In [31]:
# pip install openpyxl

In [32]:
import pandas as pd
import openpyxl

In [33]:
file_path = "C:/Users/José Roberto/Documents/Baygon Geral/Codigo/associacao-futebol/Teste_1_Jogador_Completo/"
file_name = "Sumula_Novo_Padrao.xlsx"
full_file_path = file_path+file_name

sumula = pd.read_excel(
    full_file_path,
    sheet_name=None,
    header=1,
    engine="calamine"
)

In [34]:
# sumula["Individual"] = sumula["Individual"][sumula["Individual"]["Boleiro"].notna()]
sumula["Individual"] = sumula["Individual"].query("Boleiro == Boleiro") # Mesmo efeito do notna()

sumula["Individual"][["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates",
                      "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Copa"]] = \
    sumula["Individual"][["Gols", "Assistencias", "Vitorias", "Derrotas", "Empates",
                          "Pontos no Baba", "Mensalidade", "Colaboracao", "Pontos de Copa"]].fillna(0).astype("int")
# sumula["Individual"]

In [ ]:
# (OK) Pizza deve fazer parte da aba Times, e não ser uma aba à parte (simplifica absurdamente o código)

# lista_resultados_pizza = []
# sumula["Pizza"]["Pizza"] = 1
# for col in sumula["Pizza"].columns:
#     if col.startswith("Colete"):
#         resultados_pizza = sumula["Individual"].merge(sumula["Pizza"], how="left", left_on=["Baba", "Colete"], right_on=["Baba", col])
#         lista_resultados_pizza.append(resultados_pizza)
# sumula_pizza = pd.concat(lista_resultados_pizza, ignore_index=True)
# sumula_pizza["Pizza"] = sumula_pizza["Pizza"].fillna(0).astype(int)

# sumula_pizza = sumula_pizza.drop(
#     columns=[col for col in sumula_pizza.columns if col.startswith("Colete") and col != "Colete"]
# )

# # Quando há mais de um "colete" vencedor, os resultados são duplicados, sendo necessário remover as duplicatas
# sumula_pizza = sumula_pizza.sort_values(by="Pizza", ascending=False).drop_duplicates(subset=["Baba", "Boleiro"], keep="first")

# sumula_pizza = sumula_pizza.sort_values(by=["Baba", "Colete", "Boleiro"], ascending=True).reset_index(drop=True)

In [37]:
# sumula_defesaspenalti = sumula_pizza.merge(sumula["DefesasPenaltis"], how="left", on=["Baba", "Boleiro"])
sumula_defesaspenalti = sumula["Individual"].merge(sumula["DefesasPenaltis"], how="left", on=["Baba", "Boleiro"])
sumula_defesaspenalti["Defesas de Penalti"] = sumula_defesaspenalti["Defesas de Penalti"].fillna(0).astype("int")
# sumula_defesaspenalti

In [ ]:
# Ouros (O), Gols Contra (C), Cartões (C), Pizza (P)
sumula_occp = (sumula_defesaspenalti
              .merge(sumula["Ouros"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Contra"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Cartoes"], how="left", on=["Baba", "Boleiro"])
              .merge(sumula["Times"][["Baba", "Colete", "Pizza"]], how="left", on=["Baba", "Colete"])
              )
sumula_occp[["Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho"]] = \
sumula_occp[["Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho"]].fillna(0).astype("int")
# sumula_occp

In [39]:
df_premios = sumula["Premios"].melt(id_vars=["Baba", "Premio"], value_name="Boleiro")[["Baba", "Premio", "Boleiro"]]
df_premios = df_premios.dropna(subset=["Boleiro"]).reset_index(drop=True)
df_premios["Valor"] = 1
df_premios = df_premios.pivot_table(index=["Baba", "Boleiro"], columns="Premio", values="Valor", fill_value=0)

df_premios.columns = [f"Premio {col}" if col in ["Gol", "Drible", "Defesa",
                                                 "Inacreditavel"] else col for col in df_premios.columns]
df_premios = df_premios.reset_index()

sumula_premios = sumula_occp.merge(df_premios, how="left", left_on=["Baba", "Boleiro"], right_on=["Baba", "Boleiro"])
sumula_premios = sumula_premios.fillna(0).astype({"Premio Gol":int, "Premio Drible":int, "Premio Defesa":int,
                                          "Premio Inacreditavel":int, "Bola Cheia":int, "Bola Murcha":int,
                                          "Clean Sheets":int, "Roleta":int})
# sumula_premios

In [40]:
sumula_presenca = sumula_premios.copy()
sumula_presenca["Presenca"] = 1
# sumula_presenca

In [41]:
df_primeiro_gol = sumula["PrimeiroGol"].copy()
df_primeiro_gol["Primeiro Gol"] = 1
sumula_primeiro_gol = sumula_presenca.merge(df_primeiro_gol, on=["Baba", "Boleiro"], how="left")
sumula_primeiro_gol["Primeiro Gol"] = sumula_primeiro_gol["Primeiro Gol"].fillna(0).astype(int)
# display(sumula_primeiro_gol)

# df_primeiro_gol_agrupado = df_primeiro_gol.copy().groupby("Boleiro", as_index=False).count()
df_primeiro_gol_agrupado = df_primeiro_gol.copy().groupby("Boleiro")["Baba"].count().reset_index(name="Primeiro Gol")
# df_primeiro_gol_agrupado

In [42]:
sumula_ranking = sumula_primeiro_gol.copy()
sumula_ranking["Ranking Geral"] = (
    sumula_ranking["Gols"]
    + sumula_ranking["Assistencias"]
    + sumula_ranking["Vitorias"] * 3
    + sumula_ranking["Empates"] * 3
    + sumula_ranking["Mensalidade"]
    + sumula_ranking["Colaboracao"]
    + sumula_ranking["Defesas de Penalti"]
    + sumula_ranking["Bola Cheia"] * 3
    + sumula_ranking["Clean Sheets"] * 3
    + sumula_ranking["Premio Defesa"] * 3
    + sumula_ranking["Premio Drible"] * 3
    + sumula_ranking["Premio Gol"] * 3
)

sumula_ranking["Ranking Selecao"] = sumula_ranking["Ranking Geral"] - sumula_ranking["Colaboracao"]
df_final_sumula_ranking = sumula_ranking
# df_final_sumula_ranking

In [43]:
ordem_posicoes = ["Goleiro", "Zagueiro", "Ala", "Meia", "Atacante"]
ordem_meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
    "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

In [55]:
df_boleiros = sumula["Boleiros"].copy()
df_baba = sumula["Baba"][["Baba", "Data"]].copy()

df_baba["Mes"] = df_baba["Data"].dt.month_name(locale="Portuguese_Brazil")
df_baba["Ano"] = df_baba["Data"].dt.year

df_validacao_ultimo_baba_mes = df_baba.copy()
df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes.sort_values(
    ["Ano", "Mes", "Baba"], ascending=[True, True, False]
)

df_validacao_ultimo_baba_mes["Total Babas"] = (
    df_validacao_ultimo_baba_mes.groupby(["Ano", "Mes"])["Baba"].transform("count")
)

df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes.drop_duplicates(subset=["Ano", "Mes"])[
    ["Ano", "Mes", "Baba", "Total Babas"]
]
df_validacao_ultimo_baba_mes = df_validacao_ultimo_baba_mes[["Baba", "Total Babas"]].reset_index(drop=True)


df_ultimo_baba_mes = sumula_ranking.merge(df_baba, on=["Baba"], how="left")
df_ultimo_baba_mes = df_ultimo_baba_mes.sort_values(
    ["Ano", "Mes", "Baba"], ascending=[True, True, False]
)
df_ultimo_baba_mes = df_ultimo_baba_mes.drop_duplicates(subset=["Ano", "Mes"])[
    ["Ano", "Mes", "Baba"]
]

df_ultimo_baba_mes = df_ultimo_baba_mes.rename(columns={"Baba":"Ultimo Baba"})

df_ranking_sem_colab = sumula_ranking[["Baba", "Boleiro", "Ranking Selecao", "Vitorias", "Gols", "Assistencias"]].copy()
df_pontuacao_por_mes = df_ranking_sem_colab \
                    .merge(df_boleiros, on="Boleiro", how="left") \
                    .merge(df_baba, on=["Baba"], how="left")
df_pontuacao_por_mes["Mes"] = df_pontuacao_por_mes["Data"].dt.month_name(locale="Portuguese_Brazil")
df_pontuacao_por_mes["Ano"] = df_pontuacao_por_mes["Data"].dt.year
df_pontuacao_por_mes = df_pontuacao_por_mes.groupby(["Ano", "Mes", "Boleiro", "Posicao"])[
    ["Ranking Selecao", "Vitorias", "Gols", "Assistencias"]
].sum().reset_index()

df_pontuacao_por_mes = df_pontuacao_por_mes.merge(df_ultimo_baba_mes, on=["Ano", "Mes"], how="left")

df_pontuacao_por_mes["Posicao"] = pd.Categorical(df_pontuacao_por_mes["Posicao"], categories=ordem_posicoes, ordered=True)
df_pontuacao_por_mes["Mes"] = pd.Categorical(df_pontuacao_por_mes["Mes"], categories=ordem_meses, ordered=True)

df_pontuacao_por_mes = df_pontuacao_por_mes.sort_values(["Ano", "Mes", "Posicao", "Ranking Selecao",
                                                       "Vitorias", "Gols", "Assistencias", "Boleiro"],
                                                       ascending=[False, False, True, False, 
                                                                  False, False, False, True])

# A validação deve acontecer somente na Seleção do Mês. Pontuação do Mês deve conter tudo
# df_pontuacao_por_mes = df_pontuacao_por_mes[
#     df_pontuacao_por_mes["Ultimo Baba"].isin(df_validacao_ultimo_baba_mes["Baba"])
# ]

df_pontuacao_por_mes = df_pontuacao_por_mes \
    .merge(df_validacao_ultimo_baba_mes, left_on="Ultimo Baba", right_on="Baba", how="left") \
    .drop(columns="Baba")

# df_pontuacao_por_mes

In [56]:
df_selecao_do_mes = (
    df_pontuacao_por_mes
    .copy()
    [df_pontuacao_por_mes["Total Babas"] >= 3]
    .sort_values(
        by=["Ano", "Mes", "Posicao", "Ranking Selecao", "Vitorias", "Gols", "Assistencias", "Boleiro"],
        ascending=[True, False, True, False, False, False, False, True]
    )
    .groupby(["Ano", "Mes", "Posicao"], as_index=False, observed=True)
    .head(1)
    .reset_index(drop=True)
)

df_selecao_do_mes = df_selecao_do_mes[
    df_selecao_do_mes["Ultimo Baba"].isin(df_validacao_ultimo_baba_mes["Baba"])
]

df_selecao_do_mes = df_selecao_do_mes \
    .merge(df_validacao_ultimo_baba_mes, left_on="Ultimo Baba", right_on="Baba", how="left") \
    .drop(columns="Baba")

remover_colunas = ["Vitorias", "Gols", "Assistencias"]
# ordenar_selecao_do_mes = ["Ano", "Mes", "Ultimo Baba", "Boleiro", "Posicao", "Pontuacao"]
ordenar_selecao_do_mes = ["Ano", "Mes", "Boleiro", "Posicao", "Pontuacao"]

df_selecao_do_mes = df_selecao_do_mes.rename(columns={"Ranking Selecao":"Pontuacao"}).drop(columns=remover_colunas)
df_final_selecao_do_mes = df_selecao_do_mes[ordenar_selecao_do_mes]

# df_final_selecao_do_mes

In [57]:
df_final_pontuacao_por_mes = df_pontuacao_por_mes.rename(columns={"Ranking Selecao": "Pontuacao"}).drop(columns=["Ultimo Baba", "Total Babas"])
# df_final_pontuacao_por_mes

In [47]:
# df_ultima_pizza = sumula_ranking[["Baba", "Boleiro", "Pizza"]].sort_values("Baba", ascending=False).copy()
# lista_ultima_pizza = df_ultima_pizza[df_ultima_pizza["Pizza"] == 1].groupby("Boleiro")["Baba"].first()
# df_ultima_pizza["Ultima Pizza"] = df_ultima_pizza["Boleiro"].map(lista_ultima_pizza).fillna("Não ganhou")
# df_ultima_pizza = df_ultima_pizza[["Boleiro", "Ultima Pizza"]].drop_duplicates().reset_index(drop=True)
# df_ultima_pizza

In [ ]:
df_ultima_presenca = (
    sumula_ranking[["Baba", "Boleiro"]]
    .copy()
    .sort_values("Baba", ascending=False)
    .groupby("Boleiro", as_index=False)["Baba"]
    .first()
    .rename(columns={"Baba": "Ultima Presenca"})
)
# df_ultima_presenca

,Boleiro,Ultima Presenca
0,Anderson,T10B04
1,André,T10B04
2,Carlos,T10B04
3,Henrique,T10B04
4,João,T10B03
5,Juan,T10B01
6,Júlio,T10B04
7,Leandro,T10B04
8,Lúcio,T10B04
9,Matheus,T10B04


In [ ]:
colunas_para_somar = [
    "Gols", "Assistencias", "Vitorias", "Derrotas", "Empates", "Pontos no Baba",
    "Mensalidade", "Colaboracao", "Pontos de Copa", "Pizza", "Defesas de Penalti",
    "Ouros", "Gols Contra", "Cartao Amarelo", "Cartao Azul", "Cartao Vermelho",
    "Bola Cheia", "Bola Murcha", "Clean Sheets", "Premio Defesa", "Premio Drible",
    "Premio Gol", "Premio Inacreditavel", "Roleta", "Presenca", "Ranking Geral",
    "Ranking Selecao"
]

df_ranking_agrupado = (
    sumula_ranking
    .copy()
    .groupby("Boleiro", as_index=False)[colunas_para_somar]
    .sum()
)

df_ranking_completo = (df_ranking_agrupado
                    #    .merge(df_ultima_pizza, on="Boleiro", how="left")
                       .merge(df_ultima_presenca, on="Boleiro", how="left")
                       .merge(df_primeiro_gol_agrupado, on="Boleiro", how="left")
                       .merge(df_boleiros, on="Boleiro", how="left")
                       )

df_ranking_completo["Primeiro Gol"] = df_ranking_completo["Primeiro Gol"].fillna(0).astype(int)

df_final_ranking_completo = df_ranking_completo
# df_final_ranking_completo

,Boleiro,Gols,Assistencias,Vitorias,Derrotas,Empates,Pontos no Baba,Mensalidade,Colaboracao,Pontos de Copa,...,Premio Drible,Premio Gol,Premio Inacreditavel,Roleta,Presenca,Ranking Geral,Ranking Selecao,Ultima Presenca,Primeiro Gol,Posicao
0,Anderson,10,4,13,11,5,62,0,0,20,...,0,0,0,1,4,68,68,T10B04,0,Atacante
1,André,4,1,10,6,4,42,6,10,0,...,0,0,0,0,3,63,53,T10B04,0,Meia
2,Carlos,10,6,13,11,5,64,9,0,0,...,0,1,0,1,4,85,85,T10B04,1,Ala
3,Henrique,0,1,2,2,3,11,0,0,0,...,0,0,1,1,1,16,16,T10B04,0,Ala
4,João,7,2,11,9,2,47,0,0,0,...,0,1,0,1,3,54,54,T10B03,0,Zagueiro
5,Juan,0,2,3,5,1,13,0,0,0,...,0,0,0,0,1,14,14,T10B01,0,Atacante
6,Júlio,6,6,11,13,5,54,3,0,5,...,1,0,0,1,4,66,66,T10B04,0,Meia
7,Leandro,11,3,11,13,5,56,6,0,0,...,0,1,0,0,4,74,74,T10B04,1,Zagueiro
8,Lúcio,1,0,0,0,1,3,0,0,0,...,0,0,0,0,1,4,4,T10B04,0,Meia
9,Matheus,6,5,13,11,4,58,6,30,0,...,1,0,1,1,4,117,87,T10B04,0,Goleiro


In [ ]:
df_boleiros_times = sumula["Individual"][["Baba", "Colete", "Boleiro"]]
df_times = sumula["Times"]

df_duplas = df_boleiros_times.merge(df_times, on=["Baba", "Colete"], how="left")
df_final_duplas = (
    df_duplas[["Baba", "Colete", "Boleiro"]].merge(df_duplas, on=["Baba", "Colete"])
    .query("Boleiro_x != Boleiro_y")
    .rename(columns={"Boleiro_x": "Boleiro", "Boleiro_y": "Boleiro2"})
    .reset_index(drop=True)
)
# df_final_duplas

,Baba,Colete,Boleiro,Boleiro2,Vitorias,Derrotas,Empates,Pizza
0,T10B01,Vermelho,Zé,Pedro,5,3,1,1
1,T10B01,Vermelho,Zé,Leandro,5,3,1,1
2,T10B01,Vermelho,Zé,Júlio,5,3,1,1
3,T10B01,Vermelho,Zé,Renato,5,3,1,1
4,T10B01,Vermelho,Pedro,Zé,5,3,1,1
...,...,...,...,...,...,...,...,...
165,T10B04,Branco,Lúcio,Anderson,2,2,3,1
166,T10B04,Branco,Lúcio,Carlos,2,2,3,1
167,T10B04,Branco,Lúcio,André,2,2,3,1
168,T10B04,Branco,Lúcio,Matheus,2,2,3,1


In [ ]:
grupo_duplas = df_final_duplas.groupby(["Boleiro", "Boleiro2"]).agg(
    Total_Pizza=("Pizza", "sum"),
    Total_Vitorias=("Vitorias", "sum"),
    Total_Derrotas=("Derrotas", "sum"),
    Total_Empates=("Empates", "sum"),
    Total_Babas=("Baba", "count")
).reset_index()
grupo_duplas["Total_Partidas"] = grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"] + grupo_duplas["Total_Empates"]
grupo_duplas["P_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_VD"] = grupo_duplas["Total_Vitorias"] / (grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"])
grupo_duplas["M_Pizza"] = grupo_duplas["Total_Pizza"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Babas"]
grupo_duplas["M_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Babas"]
grupo_duplas = grupo_duplas[
            [
                "Boleiro",
                "Boleiro2",
                "Total_Pizza",
                "Total_Vitorias",
                "Total_Derrotas",
                "Total_Empates",
                "Total_Partidas",
                "Total_Babas",
                "P_Vitorias",
                "P_Derrotas",
                "P_Empates",
                "P_VD",
                "M_Pizza",
                "M_Vitorias",
                "M_Derrotas",
                "M_Empates"
            ]
        ]

grupo_duplas.columns = grupo_duplas.columns.str.replace("_", " ")

df_final_grupo_duplas = grupo_duplas.sort_values(["Total Babas", "Total Partidas", "Total Vitorias"], 
                                                 ascending=False).reset_index(drop=True)
# df_final_grupo_duplas

,Boleiro,Boleiro2,Total Pizza,Total Vitorias,Total Derrotas,Total Empates,Total Partidas,Total Babas,P Vitorias,P Derrotas,P Empates,P VD,M Pizza,M Vitorias,M Derrotas,M Empates
0,Anderson,Carlos,3,13,11,5,29,4,0.448276,0.379310,0.172414,0.541667,0.75,3.25,2.75,1.25
1,Anderson,Matheus,3,13,11,5,29,4,0.448276,0.379310,0.172414,0.541667,0.75,3.25,2.75,1.25
2,Carlos,Anderson,3,13,11,5,29,4,0.448276,0.379310,0.172414,0.541667,0.75,3.25,2.75,1.25
3,Carlos,Matheus,3,13,11,5,29,4,0.448276,0.379310,0.172414,0.541667,0.75,3.25,2.75,1.25
4,Matheus,Anderson,3,13,11,5,29,4,0.448276,0.379310,0.172414,0.541667,0.75,3.25,2.75,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Lúcio,Carlos,1,2,2,3,7,1,0.285714,0.285714,0.428571,0.500000,1.00,2.00,2.00,3.00
62,Lúcio,Henrique,1,2,2,3,7,1,0.285714,0.285714,0.428571,0.500000,1.00,2.00,2.00,3.00
63,Lúcio,Matheus,1,2,2,3,7,1,0.285714,0.285714,0.428571,0.500000,1.00,2.00,2.00,3.00
64,Matheus,Henrique,1,2,2,3,7,1,0.285714,0.285714,0.428571,0.500000,1.00,2.00,2.00,3.00


In [52]:
# writer = pd.ExcelWriter(full_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace")
# with pd.ExcelWriter(full_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:

# Novo arquivo .xlsx porque incluir abas no antigo leva muito tempo (~48s)
new_file = r"Resultados_Ranking.xlsx"
new_full_file_path = file_path+new_file

def escrever_tabelas_no_excel(writer, dataframe, sheet_name, titulo):
    dataframe.to_excel(writer, sheet_name=sheet_name, startrow=1, index=False)
    worksheet = writer.sheets[sheet_name]
    worksheet.cell(row=1, column=1).value = titulo
    # writer.close() # Substituído pelo "with", que gerencia abertura e fechamento do arquivo, causou erro ao abrir 2ª vez
    
with pd.ExcelWriter(new_full_file_path, engine="openpyxl", mode="w") as writer:
    escrever_tabelas_no_excel(writer, df_final_sumula_ranking, "Sumula por Baba", "Sumula por Baba")
    escrever_tabelas_no_excel(writer, df_final_ranking_completo, "Ranking Completo", "Ranking Completo")
    escrever_tabelas_no_excel(writer, df_final_selecao_do_mes, "Selecao do Mes", "Seleção do Mês")
    escrever_tabelas_no_excel(writer, df_final_pontuacao_por_mes, "Pontuacao por Mes", "Pontuação por Mês")
    escrever_tabelas_no_excel(writer, df_final_grupo_duplas, "Estatisticas por Duplas", "Estatísticas por Duplas")
    escrever_tabelas_no_excel(writer, df_final_duplas, "Desempenho por Duplas", "Desempenho por Duplas")